# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233752463096                   -0.50    8.0
  2   -7.249722956985       -1.80       -1.37    1.0
  3   -7.249919394734       -3.71       -1.56    4.0
  4   -7.251031475472       -2.95       -1.91    2.0
  5   -7.251315711854       -3.55       -2.40    2.0
  6   -7.251338289206       -4.65       -3.24    1.0
  7   -7.251338694277       -6.39       -3.56    3.0
  8   -7.251338795595       -6.99       -4.21    3.0
  9   -7.251338798521       -8.53       -4.86    3.0
 10   -7.251338798679       -9.80       -5.29    3.0
 11   -7.251338798697      -10.75       -5.62    3.0
 12   -7.251338798704      -11.13       -6.48    3.0
 13   -7.251338798705      -12.85       -6.91    3.0
 14   -7.251338798705      -13.85       -7.35    2.0
 15   -7.251338798705   +    -Inf       -7.90    1.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04738517206029606
[ Info: Arnoldi iteration step 2: normres = 0.5376036025176781
[ Info: Arnoldi iteration step 3: normres = 0.8838495302316605
[ Info: Arnoldi iteration step 4: normres = 0.39317957050384494
[ Info: Arnoldi iteration step 5: normres = 0.4235838251171594
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.73e-02, 7.04e-02, 3.29e-01, 2.50e-01, 1.44e-02)
[ Info: Arnoldi iteration step 6: normres = 0.37992368237628604
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.82e-02, 1.22e-01, 3.23e-01, 1.26e-01, 8.50e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07222892439305195
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.47e-04, 1.07e-02, 1.86e-02, 3.16e-02, 5.34e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1078671612355406
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.06e-